# 機械学習をPythonで実践する-20　　～ 特徴量選択 ～

In [181]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder, OneHotEncoder
# # import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,cross_val_score, KFold, RepeatedKFold,StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from lightGBM_cv import lightGBM_classifier_cv_func
from category_encoders import TargetEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel

%matplotlib inline
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Greedy Feature Selection
Kaggleのpenguinデータセットを使ってGreedy Feature Selectionをやってみる。  
GBDTではあまり積極的に特徴量選択を行わない※ことから、モデルにはロジスティック回帰を使う。  

※GBDTはノイズに強いため、元々ある特徴量は全て使い、特徴量エンジニアリングで増やした特徴量の効果をCVで検証して採用するかどうか決めることが多い様子。  

In [39]:
dtypes = {
    "species": str,
    'island': str,
    'culmen_length_mm': pl.Float32, # くちばしの長さ[mm]
    'culmen_depth_mm': pl.Float32, # くちばしの高さ[mm]
    'flipper_length_mm': pl.Float32, # 翼の長さ[mm]
    'body_mass_g': pl.Float32, # 体重[g]
    'sex': str
}

# ペンギンのデータセット読み込み。欠損値がNAとして含まれているので、null_values="NA"を指定しないと読み込みエラーになる。
df = pl.read_csv('../Python/sample_data/ML_sample/penguins_size.csv',dtypes=dtypes, null_values='NA')


### - 前処理

In [40]:
df.null_count()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,u32,u32,u32,u32,u32,u32
0,0,2,2,2,2,10


In [41]:
# sexカラムの.は欠損値扱いとする。
df = df.with_columns(
    (pl.when(pl.col('sex') == '.').then(None).otherwise(pl.col('sex'))).alias('sex')
)

In [42]:
# 欠損値が多すぎる行（値が入っている列が3つより少ない行）を削除する。
# この操作はPolarsだと面倒なので、一回Pandasに変換してやる。
df = pl.from_pandas(df.to_pandas().dropna(thresh=3))

In [43]:
df.null_count()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,9


In [44]:
target = 'species'
X = df.drop(target)
y = df.get_column(target)

In [45]:
# sexカラムの欠損値を文字列の'NaN'で置き換える
X = X.fill_null('NaN')
X.null_count()

island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,u32,u32,u32,u32,u32
0,0,0,0,0,0


今回は欠損値を新たなカテゴリ'NaN'として扱うだけなので、欠損値対応を交差検証の中で行う必要はない。  
この段階でOne-hot Encodingによるダミー変数化を行うと、greedy feature selectionによる特徴量選択がダミー変数も含む状態で行われ、  
計算量が多くなる（多分）、かつ元の特徴量を部分的に使うことになってしまいわかりづらくなってしまうので、  
ダミー変数化はpipelineにしてgreedy feature selectionのCVの中で行う。

### - 特徴量エンジニアリング

ここでは多項式特徴量と四則演算した結果を用いる。

In [46]:
# 多項式特徴量
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_length_depth = poly.fit_transform(X.select(['culmen_length_mm', 'culmen_depth_mm']))

In [47]:
X = X.with_columns([
    pl.Series(poly_length_depth[:, 0]).alias('culmen_length_mm'),
    pl.Series(poly_length_depth[:, 1]).alias('culmen_depth_mm'),
    pl.Series(poly_length_depth[:, 2]).alias('culmen_length_mm^2'),
    pl.Series(poly_length_depth[:, 3]).alias('culmen_length_X_depth'),
    pl.Series(poly_length_depth[:, 4]).alias('culmen_depth_mm^2')
])

In [48]:
# culmen_length_mm, culmen_depth_mmの差と比を計算する
X = X.with_columns([
    (pl.col('culmen_length_mm') - pl.col('culmen_depth_mm')).alias('culmen_diff'),
    (pl.col('culmen_length_mm') / pl.col('culmen_depth_mm')).alias('culmen_ratio'),
])

In [49]:
X.head(3)

island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex,culmen_length_mm^2,culmen_length_X_depth,culmen_depth_mm^2,culmen_diff,culmen_ratio
str,f64,f64,f32,f32,str,f64,f64,f64,f64,f64
"""Torgersen""",39.099998,18.700001,181.0,3750.0,"""MALE""",1528.809881,731.170001,349.690029,20.399998,2.090909
"""Torgersen""",39.5,17.4,186.0,3800.0,"""FEMALE""",1560.25,687.299985,302.759987,22.1,2.270115
"""Torgersen""",40.299999,18.0,195.0,3250.0,"""FEMALE""",1624.089939,725.399986,324.0,22.299999,2.238889


### - CVの準備

In [50]:
# cvインスタンスを生成。3 fold で評価する。
cv = KFold(n_splits=3, random_state=0, shuffle=True)

In [54]:
# # 一連の学習・予測用のPipelineを定義。
# pipe = Pipeline(steps=[('dummies', OneHotEncoder(drop='first',handle_unknown='ignore', sparse_output=False)),
#                        ('std_scale', StandardScaler()),
#                        ('model', LogisticRegression())])

OneHotEncoderはfitに入れたデータをすべてカテゴリカル変数としてエンコーディングしてしまうので、  
本来のカテゴリカル変数だけエンコーディングしたい場合はColumnsTransformerでカラム名のリストを指定する必要がある。  
しかし、今回の場合はGreedy Feature Selectionでカラムそれぞれに対して学習していくため、  
カラム名のリストを指定すると、学習時に該当カラムがなくてエラーになってしまう。  
そこで、自動的にカテゴリカル変数を検出可能なpandasのpd.get_dummiesを用いる。  
ただし、get_dummiesには.fitがないので、Pipelineのtransformerとして指定するには、  
新たにダミー変数化するクラスを作ってfitメソッドを実装する必要がある。  
このように既存の機能＋αのクラスを自作するケースは実務でも時々あるらしい。  

In [158]:
# ダミー変数化するクラスの実装。コア部分はpd.get_dummiesを使用している。

# Transformerを自作するには下記の２クラスを継承する必要がある。 
from sklearn.base import BaseEstimator, TransformerMixin

class GetDummies(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = None

    
    def fit(self, X, y=None):
        # 学習データをダミー変数化したときのカラム名を記録しておく
        # これは学習時に使ったカラムとテストデータに対する予測時に使うカラムを揃えるための前処理。
        self.columns = pd.get_dummies(X, drop_first=True).columns
        return self

    def transform(self, X):
        X_new = pd.get_dummies(X, drop_first=True)
        # .reindexによってX_newをcolumnsの順に並び替える。
        # もしもX_newにないカラムがある場合は、columnsに従って新しく列が追加されて全て0で埋められる。
        # これは学習データにあってテストデータにないカテゴリがある場合を想定した措置。
        # reindexの説明: https://note.nkmk.me/python-pandas-reindex/
        return X_new.reindex(columns=self.columns, fill_value=0)


In [159]:
gd = GetDummies()
gd.fit(X.to_pandas())
gd.transform(X.to_pandas()).head()

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,culmen_length_mm^2,culmen_length_X_depth,culmen_depth_mm^2,culmen_diff,culmen_ratio,island_Dream,island_Torgersen,sex_MALE,sex_NaN
0,39.099998,18.700001,181.0,3750.0,1528.809881,731.170001,349.690029,20.399998,2.090909,0,1,1,0
1,39.500000,17.400000,186.0,3800.0,1560.250000,687.299985,302.759987,22.100000,2.270115,0,1,0,0
2,40.299999,18.000000,195.0,3250.0,1624.089939,725.399986,324.000000,22.299999,2.238889,0,1,0,0
3,36.700001,19.299999,193.0,3450.0,1346.890056,708.309987,372.489971,17.400002,1.901555,0,1,0,0
4,39.299999,20.600000,190.0,3650.0,1544.489940,809.579999,424.360016,18.699999,1.907767,0,1,1,0


ちゃんと動いていそう

In [160]:
# Pipelineの定義
pipe = Pipeline([('dummiy', GetDummies()),
                 ('scaler', StandardScaler()),
                 ('model', LogisticRegression())])

pipe

Pipeline(steps=[('dummiy', GetDummies()), ('scaler', StandardScaler()),
                ('model', LogisticRegression())])

### - Greedy feature selectionクラスの定義

In [161]:
class GreedyFeatureSelection():

    def __init__(self, pipeline, cv) -> None:
        self.pipeline = pipeline
        self.cv = cv
        self.selected_features = []
        self.scores = {}
    
    def select(self, X, y):
        # 初期化
        best_score_temp = 0
        best_score_feature_temp = ''
        # X.columnsはindexオブジェクトを返すため、リストにしておかないとremove等は使えない。
        features_candidate = list(X.columns)
        max_select_iterations = len(X.columns)
        i = 0
        
        # 特徴量の選定ループ。whileループ1回につき、最大1つ特徴量が選定される。
        while i != max_select_iterations:

            # 「選定済み特徴量に特徴量を１つ加えて評価」を残り全特徴量分実施
            for feature in features_candidate:
                # 学習に使用する特徴量を定義
                use_features = [feature] + self.selected_features
                # 使用した特徴量によるCV評価結果の平均値をスコアとする。
                score = cross_val_score(estimator=self.pipeline, cv=self.cv, X=X[use_features], y=y).mean()
                # 暫定最高スコアと上記スコアの比較をし、高ければ暫定最高スコアを更新し、その特徴量を暫定チャンピオンとして格納
                if best_score_temp < score:
                    best_score_temp = score
                    best_score_feature_temp = feature

            # best_score_feature_temp（上記ループでの最終的なチャンピオン特徴量）が特徴量候補にある場合、
            # チャンピオン特徴量を特徴量候補から除き、選定した特徴量のリストに加える
            # 候補にない場合は、チャンピオン特徴量に更新がないということなので、選定を終了する。
            if best_score_feature_temp in features_candidate:
                features_candidate.remove(best_score_feature_temp)
                self.selected_features.append(best_score_feature_temp)
                self.scores[best_score_feature_temp] = best_score_temp
                i += 1
                print(f'特徴量{i}: {best_score_feature_temp}, スコア:{best_score_temp}')
            else:
                break

        print('Greedy Feature Selection done!')


In [162]:
gfs = GreedyFeatureSelection(pipe, cv)
gfs.select(X.to_pandas(), y.to_pandas())

特徴量1: culmen_ratio, スコア:0.9619883040935672
特徴量2: island, スコア:0.9912280701754387
特徴量3: culmen_depth_mm^2, スコア:0.9970760233918128
特徴量4: body_mass_g, スコア:1.0
Greedy Feature Selection done!


In [165]:
gfs.scores

{'culmen_ratio': 0.9619883040935672,
 'island': 0.9912280701754387,
 'culmen_depth_mm^2': 0.9970760233918128,
 'body_mass_g': 1.0}

上記のような結果となった。  
特徴量エンジニアリングの段階で有効そうだったculmen_ratioが一番に選ばれており、ちゃんと実装できていそう。

## Recursive Feature Elimination(RFE)

In [169]:
# RFE用のモデルには今回決定木を用いる
model = tree.DecisionTreeClassifier()

# 残す特徴量の数を指定。estimatorには決定木のように重要度を計算できるモデルでないとだめ。
rfe = RFE(estimator=model, n_features_to_select=6)

In [170]:
# rfeにfitしてtransformする。
X_pd = pd.get_dummies(X.to_pandas(), drop_first=True)
rfe.fit(X_pd, y)
rfe.set_output(transform='pandas')
rfe.transform(X_pd)

,culmen_diff,culmen_ratio,island_Dream,island_Torgersen,sex_MALE,sex_NaN
0,20.399998,2.090909,0.0,1.0,1.0,0.0
1,22.100000,2.270115,0.0,1.0,0.0,0.0
2,22.299999,2.238889,0.0,1.0,0.0,0.0
3,17.400002,1.901555,0.0,1.0,0.0,0.0
4,18.699999,1.907767,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...
337,33.500001,3.445256,0.0,0.0,0.0,0.0
338,32.499999,3.272727,0.0,0.0,0.0,0.0
339,34.700002,3.210191,0.0,0.0,1.0,0.0
340,30.400001,3.054054,0.0,0.0,0.0,0.0


特徴量選択された結果が返される。

In [172]:
# 重要度ランキングを表示
rfe.ranking_

array([8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1])

In [173]:
# ランキング１位のものが選択結果になっている
rfe.feature_names_in_[rfe.ranking_==1]

array(['culmen_diff', 'culmen_ratio', 'island_Dream', 'island_Torgersen',
       'sex_MALE', 'sex_NaN'], dtype=object)

In [174]:
rfe.support_

array([False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True])

In [175]:
rfe.feature_names_in_[rfe.support_]

array(['culmen_diff', 'culmen_ratio', 'island_Dream', 'island_Torgersen',
       'sex_MALE', 'sex_NaN'], dtype=object)

.supportは選ばれたかどうかのマスクが返される。  
実際の学習にrfeを組み込む場合は、パイプラインの中でrfeをすればよい。  
(rfeのestimatorと学習に使うestimatorは同じ方が良い気がする)

In [179]:
# RFEをCVで回す場合,RFECVを使う。RFECVでは自動的に特徴量の数が決定されるので指定する必要はない。
rfecv = RFECV(model, cv=cv)
rfecv.fit(X_pd, y)
rfecv.set_output(transform='pandas')
rfecv.transform(X_pd)

,culmen_diff,culmen_ratio,island_Dream
0,20.399998,2.090909,0.0
1,22.100000,2.270115,0.0
2,22.299999,2.238889,0.0
3,17.400002,1.901555,0.0
4,18.699999,1.907767,0.0
...,...,...,...
337,33.500001,3.445256,0.0
338,32.499999,3.272727,0.0
339,34.700002,3.210191,0.0
340,30.400001,3.054054,0.0


In [180]:
rfecv.feature_names_in_[rfecv.support_]

array(['culmen_diff', 'culmen_ratio', 'island_Dream'], dtype=object)

GreedyFeatureSelectionの結果と似たような感じになった。

## 特徴量の重要度をモデルの教えてもらって特徴量選択

In [193]:
# モデルからの特徴量選択用インスタンスを生成。thresholdは選択するか否かの重要度の閾値
sfm = SelectFromModel(estimator=RandomForestClassifier(random_state=0), threshold='median')
sfm.set_output(transform='pandas')
X_selected = sfm.fit_transform(X_pd, y)

In [195]:
X_selected.head(3)

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,culmen_length_mm^2,culmen_depth_mm^2,culmen_diff,culmen_ratio
0,39.099998,18.700001,181.0,1528.809881,349.690029,20.399998,2.090909
1,39.500000,17.400000,186.0,1560.250000,302.759987,22.100000,2.270115
2,40.299999,18.000000,195.0,1624.089939,324.000000,22.299999,2.238889


In [189]:
# 重要度を表示
print(sfm.feature_names_in_)
print(sfm.estimator_.feature_importances_)

['culmen_length_mm' 'culmen_depth_mm' 'flipper_length_mm' 'body_mass_g'
 'culmen_length_mm^2' 'culmen_length_X_depth' 'culmen_depth_mm^2'
 'culmen_diff' 'culmen_ratio' 'island_Dream' 'island_Torgersen' 'sex_MALE'
 'sex_NaN']
[0.08924232 0.07924614 0.09930585 0.0525517  0.08097603 0.01473159
 0.10925035 0.14531745 0.24560675 0.07466197 0.00589045 0.0032194
 0.        ]


In [196]:
# 選ばれた特徴量を表示
sfm.feature_names_in_[sfm.get_support()]

array(['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm',
       'culmen_length_mm^2', 'culmen_depth_mm^2', 'culmen_diff',
       'culmen_ratio'], dtype=object)